In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Kaggle hub allows to download directly from kaggle instead of downloading it manuallly**

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("javohirtoshqorgonov/noise-audio-data")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/noise-audio-data


****

In [82]:
audio_path = None
for root, dirs, files in os.walk(path):
    for file in files:
        if file.lower().endswith(".wav"):
            audio_path = os.path.join(root, file)
            break
    if audio_path:
        break

if audio_path is None:
    raise FileNotFoundError("❌ No WAV file found in dataset!")

print("\n🎵 Loaded audio file:", audio_path)


🎵 Loaded audio file: /kaggle/input/noise-audio-data/ESC-50-master/audio/5-257349-A-15.wav


<h1 style="font-size:30px; font-weight:bold;">Loading the audio and resampling in 16000hz,The signal represents the  sample of audio waveform  representing the air pressure or loudnesss at the moment.because sound is vibration.In the below 80000 samples represent the amplititude value.Frequency represent how many fast the vibration happen in one second</h1>


<h1 style="font-size:30px; font-weight:bold;">Zero Crossing Rate  check how many times the  signal changes the sign(positive->negative->neutral).It find the noise level of the audio
Spectral flatness tells about the flat frequency (noise) and shaped frequency(normal)</h1>




<h1 style="font-size:30px; font-weight:bold;">The MFCC vector is a compact numerical summary of the audio signal, capturing how its frequency content behaves. Instead of using the full waveform, which may contain thousands of values, the MFCC vector reduces the sound into around 20 meaningful numbers that describe the tone, energy, and shape of the audio. For example, a clean speech clip and a noisy fan sound will produce very different MFCC vectors: speech might give something like [-210, 95, 42, -15, ...] while fan noise might give [ -80, 10, 5, -2, ... ]. These numbers act like the “fingerprint” of the sound, making it easier for machine learning models to analyze and classify audio.</h1>h1>

# 🔊 Noise Levels Before and After Cleaning

### **Noise Before Cleaning**
Before applying any noise-reduction techniques, the audio was analyzed using:

- **Zero-Crossing Rate (ZCR)** – measures impulse noise (clicks, crackles, sharp transitions)
- **Spectral Flatness** – measures how “noise-like” the frequency content is

The noisy audio produced:

- **ZCR Before Cleaning:** `0.520`  
- **Spectral Flatness Before Cleaning:** `0.127`

🔎 **Interpretation**  
- ZCR = 0.520 → extremely high impulse noise  
- Flatness = 0.127 → contains random noise components, but not fully flat  
- Overall: **The audio was heavily noisy and unstable**

---

### **Noise After Cleaning (Updated Pipeline)**  
After applying the improved noise-reduction pipeline:

- Soft spectral gating  
- VAD (voice activity–based noise removal)  
- Envelope smoothing  
- Band-pass filtering (speech band only)  
- Pre-emphasis  

The new noise metrics were:

- **ZCR After Cleaning:** `0.318`  
- **Spectral Flatness After Cleaning:** `0.004`

🔎 **Interpretation**  
- Flatness dropped from 0.127 → **0.004**  
  - ✔ Frequency noise removed successfully  
- ZCR dropped from 0.520 → **0.318** (still high)  
  - ❗ Impulse noise remains (sharp waveform spikes)

---

### ⭐ Summary
| Metric | Before Cleaning | After Cleaning | Status |
|-------|-----------------|----------------|--------|
| **ZCR** | 0.520 | 0.318 | 🔴 Still noisy (impulse noise) |
| **Flatness** | 0.127 | 0.004 | 🟢 Great improvement |

**Conclusion:**  
The updated noise-reduction pipeline removed most *frequency noise* effectively, but the audio still contains **impulse noise**, which keeps ZCR high.  
Additional smoothing or reconstruction may be required to fully reduce ZCR.



In [83]:
import librosa
import numpy as np


In [84]:
# ======================================================
# Noise Measurement Function
# ======================================================

def measure_noise(y, sr):
    """Return ZCR and Spectral Flatness."""
    zcr = float(np.mean(librosa.feature.zero_crossing_rate(y)))
    flat = float(np.mean(librosa.feature.spectral_flatness(y=y)))
    return zcr, flat


In [85]:
# ======================================================
# Noise Reduction Functions
# ======================================================

# 1) Soft Spectral Gating
def soft_spectral_gate(y, sr, softness=0.5):
    S = librosa.stft(y)
    mag, phase = librosa.magphase(S)

    noise_est = np.percentile(mag, 20, axis=1)[:, None]
    gain = 1 - softness * (noise_est / (mag + 1e-8))
    gain = np.clip(gain, 0, 1)

    cleaned_mag = mag * gain
    return librosa.istft(cleaned_mag * phase)


# 2) Voice Activity Detection
def apply_vad(y, threshold=0.02):
    env = np.abs(y)
    mask = env > threshold
    return y[mask]


# 3) Envelope Smoothing (reduces impulse noise → reduces ZCR)
def smooth_envelope(y, kernel=150):
    kernel = np.ones(kernel) / kernel
    return np.convolve(y, kernel, mode='same')


# 4) Band-pass filter (keep only speech: 200–3800 Hz)
def simple_band_filter(y, sr, low=200, high=3800):
    S = librosa.stft(y)
    freqs = librosa.fft_frequencies(sr=sr)

    mask = (freqs >= low) & (freqs <= high)
    S_filtered = S.copy()
    S_filtered[~mask, :] = 0

    return librosa.istft(S_filtered)


In [86]:
# ======================================================
# 1) NOISE MEASUREMENT BEFORE CLEANING
# ======================================================

zcr_before, flat_before = measure_noise(signal, sr)

print("\n===============================")
print("🔍 NOISE BEFORE CLEANING")
print("===============================")
print(f"ZCR Before Cleaning      = {zcr_before:.3f}")
print(f"Flatness Before Cleaning = {flat_before:.3f}")

if zcr_before > 0.08 or flat_before > 0.25:
    print("⚠️ Audio is noisy! Starting noise reduction...\n")
else:
    print("✅ Audio appears clean.\n")



🔍 NOISE BEFORE CLEANING
ZCR Before Cleaning      = 0.520
Flatness Before Cleaning = 0.127
⚠️ Audio is noisy! Starting noise reduction...



In [87]:
# ======================================================
# 2) APPLY UPDATED NOISE REDUCTION PIPELINE
# ======================================================

print("🎛 STEP 1 — Soft Spectral Gating...")
step1 = soft_spectral_gate(signal, sr, softness=0.6)

print("🎛 STEP 2 — Voice Activity Detection...")
step2 = apply_vad(step1, threshold=0.01)

print("🎛 STEP 3 — Envelope Smoothing...")
step3 = smooth_envelope(step2, kernel=150)

print("🎛 STEP 4 — Band-Pass Filter (200–3800 Hz)...")
step4 = simple_band_filter(step3, sr)

print("🎛 STEP 5 — Pre-emphasis...")
clean_signal = librosa.effects.preemphasis(step4)

print("👉 Cleaned Signal Shape:", clean_signal.shape)


🎛 STEP 1 — Soft Spectral Gating...
🎛 STEP 2 — Voice Activity Detection...
🎛 STEP 3 — Envelope Smoothing...
🎛 STEP 4 — Band-Pass Filter (200–3800 Hz)...
🎛 STEP 5 — Pre-emphasis...
👉 Cleaned Signal Shape: (5120,)


In [88]:
# ======================================================
# 3) NOISE MEASUREMENT AFTER CLEANING
# ======================================================

zcr_after, flat_after = measure_noise(clean_signal, sr)

print("\n===============================")
print("🔍 NOISE AFTER CLEANING")
print("===============================")
print(f"ZCR After Cleaning       = {zcr_after:.3f}")
print(f"Flatness After Cleaning  = {flat_after:.3f}")

print("\n📊 NOISE COMPARISON")
print("-------------------------------")
print(f"ZCR:      {zcr_before:.3f}  →  {zcr_after:.3f}")
print(f"Flatness: {flat_before:.3f}  →  {flat_after:.3f}")

if zcr_after < zcr_before:
    print("🟢 ZCR improved (lower impulse noise).")
else:
    print("🔴 ZCR still high (impulse noise remains).")

if flat_after < flat_before:
    print("🟢 Flatness improved (frequency noise reduced).")
else:
    print("🔴 Flatness still high.")

print()



🔍 NOISE AFTER CLEANING
ZCR After Cleaning       = 0.287
Flatness After Cleaning  = 0.003

📊 NOISE COMPARISON
-------------------------------
ZCR:      0.520  →  0.287
Flatness: 0.127  →  0.003
🟢 ZCR improved (lower impulse noise).
🟢 Flatness improved (frequency noise reduced).



In [89]:
# ======================================================
# 4) EXTRACT MFCC
# ======================================================

print("🎚 Extracting MFCC features...")
mfcc = librosa.feature.mfcc(y=clean_signal, sr=sr, n_mfcc=20)
print("MFCC Shape:", mfcc.shape)


🎚 Extracting MFCC features...
MFCC Shape: (20, 11)


In [90]:
# ======================================================
# 5) FINAL MFCC VECTOR
# ======================================================

mfcc_vector = np.mean(mfcc, axis=1)

print("\n===============================")
print("📌 FINAL MFCC VECTOR (20 VALUES)")
print("===============================")
print(mfcc_vector)
print("Vector Length:", len(mfcc_vector))

print("\n✅ PROCESSING COMPLETED SUCCESSFULLY!")



📌 FINAL MFCC VECTOR (20 VALUES)
[-775.92992008   29.3317769  -169.15368794   57.26109537  -28.74953188
  -47.70039788   11.69135871  -42.15151066   -1.2696862   -13.41760382
  -28.94142762   11.84617809  -21.48338016   -5.81580769    4.45712434
  -16.24117458   14.22498151   -2.83507312   -7.6691       11.92107023]
Vector Length: 20

✅ PROCESSING COMPLETED SUCCESSFULLY!
